In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout, Bidirectional, LSTM, Layer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import sys
import warnings
warnings.filterwarnings("ignore")
import os
import joblib
sys.path.append(os.path.abspath('../src'))
from data_tokenizer import procesar_texto
import data_loader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, recall_score, precision_score
from keras.layers import Dense, LSTM
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def division_datos(tfidf, mensajes=True):
    '''
    Carga y segmenta los datos en entrenamiento, validacion y prueba dado el tokenizador escogido.
    Guarda el scaler para normalizar datos en la prediccion.
    Argumentos:
        * tfidf: True para usar TF-IDF, False para usar CountVectorizer
        * mensajes: Por default es True. Indica si se desea imprimir un diagnostico de cantidad de filas y primeros registros de las bases finales

    Retorno:
        * x_train, x_test, x_val
        * y_train, y_test, y_val
    '''
    # Cargando datos de acuerdo a tokenizador seleccionado
    if tfidf:
        vectorizador = joblib.load('C:/Users/gerb2/Documents/DEEPLEARNING/taller2_tweets/Modelo_Sentimientos/models/vectorizador_tfidf.pkl')
        x = joblib.load('C:/Users/gerb2/Documents/DEEPLEARNING/taller2_tweets/Modelo_Sentimientos/models/tweets_tfidf.pkl')
    else:
        vectorizador = joblib.load('C:/Users/gerb2/Documents/DEEPLEARNING/taller2_tweets/Modelo_Sentimientos/models/vectorizador_tf.pkl')
        x = joblib.load('C:/Users/gerb2/Documents/DEEPLEARNING/taller2_tweets/Modelo_Sentimientos/models/tweets_tf.pkl')
    
    y = joblib.load('C:/Users/gerb2/Documents/DEEPLEARNING/taller2_tweets/Modelo_Sentimientos/models/labels.pkl')
    
    # Division en train, test y validacion
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

    if mensajes:
        print("Dimensiones de X completa:", x.shape)
        print("Dimensiones de X train:", x_train.shape)
        print("Dimensiones de X test:", x_test.shape)

        print("\nPrimeros registros X test:")
        print(x_test.toarray()[:5])

        print("\nPrimeras 5 etiquetas")
        print(y[:5])
    
    return x_train, x_test, x_val, y_train, y_test, y_val

In [ ]:
# Definir parámetros del vocabulario y la secuencia
max_features = 10000
maxlen  = 130

In [ ]:
# Llamada correcta a la función division_datos
x_train, x_test, x_val, y_train, y_test, y_val = division_datos(tfidf=True)

In [ ]:
#Modelo BiLSTM NO EJECUTAR

model = Sequential([
    Embedding(input_dim=max_features, output_dim=128, input_length=maxlen),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
    ])

In [ ]:
class Attention(Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features): # Only one input is needed for the Attention layer
        # hidden_with_time_axis = tf.expand_dims(hidden, 1) # Removed, as hidden state is not needed

        # Calculate attention weights based only on features
        score = tf.nn.tanh(self.W1(features))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector # Return only the context vector



# Modelo BiLSTM con atención
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=128, input_length=maxlen))
model.add(Bidirectional(LSTM(64, return_sequences=True)))  # return_sequences=True para la atención
model.add(Attention(64))  # Capa de atención
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2)

In [ ]:
Epoch 1/3
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'attention', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
280/280 ━━━━━━━━━━━━━━━━━━━━ 109s 372ms/step - accuracy: 0.9234 - loss: 0.2486 - val_accuracy: 0.9519 - val_loss: 0.1402
Epoch 2/3
280/280 ━━━━━━━━━━━━━━━━━━━━ 138s 359ms/step - accuracy: 0.9638 - loss: 0.1017 - val_accuracy: 0.9564 - val_loss: 0.1245
Epoch 3/3
280/280 ━━━━━━━━━━━━━━━━━━━━ 142s 358ms/step - accuracy: 0.9841 - loss: 0.0546 - val_accuracy: 0.9580 - val_loss: 0.1376

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f'Test Accuracy: {acc}, test Loss:{loss}')

In [ ]:
Test Accuracy: 0.9570892453193665, test Loss:0.1369503140449524

In [ ]:
#Ver métricas más completas (confusion matrix, precision, recall, F1)

# Predicciones
y_pred = model.predict(x_test)
y_pred_classes = (y_pred > 0.5).astype("int32")

# Reporte
print(classification_report(y_test, y_pred_classes))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión")
plt.show()

In [ ]:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5203
           1       0.77      0.55      0.64       390

    accuracy                           0.96      5593
   macro avg       0.87      0.77      0.81      5593
weighted avg       0.95      0.96      0.95      5593

In [ ]:
# Visualizar el entrenamiento

plt.plot(history.history['accuracy'], label='Accuracy entrenamiento')
plt.plot(history.history['val_accuracy'], label='Accuracy validación')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.title('Precisión durante el entrenamiento')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Loss entrenamiento')
plt.plot(history.history['val_loss'], label='Loss validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Pérdida durante el entrenamiento')
plt.legend()
plt.show()